# Miller Classification Midterm
**Author:** Dan Miller

**Date:** November 5th, 2025

**Objective:** To successfully create a ML classification model on the UCI Mushroom dataset that can predict if a mushroom is poisonous or not

## Introduction

This project explores the UCI Mushroom dataset.  The dataset contains categorical features that describe mushrooms, and we want to be able to predict, based off of certain input features, if a mushroom is poisonous or not.  After exploring the dataset, input features will be chosen and a classification model will be made.  

## Section 1. Import and Inspect the Data

### 1.1 Import the necessary libraries and load the dataset

In [2]:
# Imports

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [3]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data"

columns = [
    "class", "cap-shape", "cap-surface", "cap-color", "bruises", "odor",
    "gill-attachment", "gill-spacing", "gill-size", "gill-color",
    "stalk-shape", "stalk-root", "stalk-surface-above-ring", "stalk-surface-below-ring",
    "stalk-color-above-ring", "stalk-color-below-ring", "veil-type", "veil-color",
    "ring-number", "ring-type", "spore-print-color", "population", "habitat"
]

mushrooms = pd.read_csv(url, header=None, names=columns)

# Display the first few rows
mushrooms.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
